In [1]:
include("../MyReverseDiff.jl")
include("../MyMlp.jl")

using .MyReverseDiff
using .MyMlp
using JLD2

## Przygotowanie danych IMDB

In [2]:
X_train = Matrix(load("../../dataset/imdb_dataset_prepared.jld2", "X_train"));
y_train = Matrix(load("../../dataset/imdb_dataset_prepared.jld2", "y_train"));
X_test = Matrix(load("../../dataset/imdb_dataset_prepared.jld2", "X_test"));
y_test = Matrix(load("../../dataset/imdb_dataset_prepared.jld2", "y_test"));

input_size = size(X_train, 1) # Liczba cech

17703

##  Trening modelu

In [3]:
batch_size = 64

model = Chain(
    ConvolutionBlock(3,3,3,name="layer1"),
    ConvolutionBlock(3,3,3,name="layer2"),
    FlattenBlock(name="flatten"),
    Dense(input_size, 1, identity, name="softnet")
)

#   Utworzenie początkowych węzłów Constant dla danych wejściowych i etykiet
x_input_node = Constant(zeros(Float32, input_size, batch_size))
y_label_node = Constant(zeros(Float32, 1, batch_size))

#   Budowanie grafu treningowego
loss_node, model_output_node, order = build_graph!(model, binarycrossentropy, x_input_node, y_label_node; loss_name="loss")

optimizer_state = setup_optimizer(Adam(), model)


epochs = 5


for epoch in 1:epochs
    # --- Tasowanie zbioru treningowego NA NOWO w każdej epoce ---
    permutation = randperm(size(X_train, 2))
    X_train_shuffled_epoch = X_train[:, permutation]
    y_train_shuffled_epoch = y_train[:, permutation]
    num_batches = ceil(Int, size(X_train, 2) / batch_size)

    loss_value = 0.0

    t = @elapsed begin

    for i in 1:num_batches

        start_idx = (i - 1) * batch_size + 1
        end_idx = min(i * batch_size, size(X_train, 2))
        x_batch = X_train_shuffled_epoch[:, start_idx:end_idx]
        y_batch = y_train_shuffled_epoch[:, start_idx:end_idx]

        current_batch_size = size(x_batch, 2)
        view(x_input_node.output, :, 1:current_batch_size) .= x_batch
        view(y_label_node.output, :, 1:current_batch_size) .= y_batch


        forward!(order)

        backward!(order)

        step!(optimizer_state)
        loss_value += loss_node.output

    end
end
    avg_loss_epoch = loss_value / num_batches

    println(@sprintf("Epoch: %d (%.2fs) \tTrain: (l: %.2f)", epoch, t, avg_loss_epoch))
end

LoadError: LoadError: UndefVarError: `@sprintf` not defined in `Main`
Suggestion: check for spelling errors or missing imports.
Hint: a global variable of this name may be made accessible by importing Printf in the current active module Main
in expression starting at c:\Users\RetailAdmin\Desktop\AWiD\MyMlp\src\notebooks\jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_W1sZmlsZQ==.jl:57